In [ ]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120

In [ ]:
import findspark
findspark.init()

In [ ]:
from source.neighbor_based_recommender import NeighborBasedRecommender
import pandas as pd
import pickle

In [ ]:
result_dict = []
for sample_size in [50, 100, 150]:
    recommender = NeighborBasedRecommender(sample_size*1000)
    recommender.fit(neighbor_size=20, recalculate=True)
    train_runtime = recommender.runtime
    rmse_train, _, _ = recommender.RMSE(df='train')
    rmse_test, rmse_dist_user, rmse_dist_movie = recommender.RMSE(df='test')
    experiment_title = f'NB_{sample_size}_Dist_RMSE_User'
    with open(f'data/results/{experiment_title}.pkl', 'wb') as f:
        pickle.dump([rmse_dist_user, experiment_title], f)
    experiment_title = f'NB_{sample_size}_Dist_RMSE_Movie'
    with open(f'data/results/{experiment_title}.pkl', 'wb') as f:
        pickle.dump([rmse_dist_movie, experiment_title], f)
    top_k_precision_train, _ = recommender.top_k_precision(df='train', k=5)
    top_k_precision_test, top_k_user = recommender.top_k_precision(df='test', k=5)
    experiment_title = f'NB_{sample_size}_Dist_Precision_Movie'
    with open(f'data/results/{experiment_title}.pkl', 'wb') as f:
        pickle.dump([top_k_user, experiment_title], f)
    hyper_result = []
    for neighbor_s in range(1, 31):
        recommender.fit(neighbor_size=neighbor_s, recalculate=False)
        top_k_precision, _ = recommender.top_k_precision(df='test', k=5)
        rmse_test, _, _ = recommender.RMSE(df='test')
        hyper_result.append([neighbor_s, rmse_test, top_k_precision])
    n_size, n_rmse, n_t_k_precision = zip(*hyper_result)
    experiment_title = f'NB_{sample_size}_Hyper_RMSE'
    result = [experiment_title, pd.Series(n_size), pd.Series(n_rmse)]
    with open(f'data/results/{experiment_title}.pkl', 'wb') as f:
        pickle.dump(result, f)
    experiment_title = f'NB_{sample_size}_Hyper_Precision'
    result = [experiment_title, pd.Series(n_size), pd.Series(n_t_k_precision)]
    with open(f'data/results/{experiment_title}.pkl', 'wb') as f:
        pickle.dump(result, f)
    result_dict.append({'sample_size': sample_size,
                        'runtime': train_runtime,
                        'rmse_train': rmse_train,
                        'rmse_test': rmse_test,
                        'top_k_precision_train': top_k_precision_train,
                        'top_k_precision_test': top_k_precision_test})

In [ ]:
result = pd.DataFrame(result_dict)
result = result[['sample_size', 'runtime', 'rmse_train', 'rmse_test',
                 'top_k_precision_train', 'top_k_precision_test']]
experiment_title = f'result'
with open(f'data/results/{experiment_title}.pkl', 'wb') as f:
    pickle.dump(result, f)